# Sampler Testing
This notebook use to test which sampler works best for the given dataset.

In [1]:
import pandas as pd

from stroke_prediction.config import PROCESSED_DATA_DIR

train_data = pd.read_parquet(PROCESSED_DATA_DIR / "train-stroke-data.parquet")
test_data = pd.read_parquet(PROCESSED_DATA_DIR / "test-stroke-data.parquet")

2025-05-27 14:04:01.638 | INFO     | stroke_prediction.config:<module>:11 - PROJ_ROOT path is: E:\Projects\stroke-prediction-api


In [2]:
from sklearn.model_selection import train_test_split

X_train_val = train_data.drop(columns=["stroke"])
y_train_val = train_data["stroke"]

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val
)

X_test = test_data.drop(columns=["stroke"])
y_test = test_data["stroke"]

We will track the experiments using MLflow.

In [ ]:
import mlflow

from stroke_prediction.util import get_or_create_experiment

mlflow.set_tracking_uri("https://mlflow.spikehub.win/")
experiment_id = get_or_create_experiment("Sampler Testing")

Before running the experiment, we will prepare the objective function that will be use to try each sampler.

In [ ]:
from typing import Callable

import optuna
from imblearn.base import BaseSampler
from imblearn.over_sampling import (
    ADASYN,
    SMOTE,
    SMOTEN,
    SVMSMOTE,
    BorderlineSMOTE,
    RandomOverSampler,
)
from imblearn.under_sampling import (
    AllKNN,
    CondensedNearestNeighbour,
    EditedNearestNeighbours,
    NearMiss,
    NeighbourhoodCleaningRule,
    OneSidedSelection,
    RandomUnderSampler,
    RepeatedEditedNearestNeighbours,
    TomekLinks,
)
from sklearn.base import BaseEstimator
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier


class DummySampler(BaseSampler):
    def __init__(self):
        super().__init__()

    def _fit_resample(self, X, y):
        return X, y

    def fit_resample(self, X, y):
        return self._fit_resample(X, y)


oversampler = {
    "ADASYN": ADASYN,
    "SMOTE": SMOTE,
    "SMOTEN": SMOTEN,
    "SVMSMOTE": SVMSMOTE,
    "BorderlineSMOTE": BorderlineSMOTE,
    "RandomOverSampler": RandomOverSampler,
    "None": DummySampler,
}

undersampler = {
    "AllKNN": AllKNN,
    "CondensedNearestNeighbour": CondensedNearestNeighbour,
    "EditedNearestNeighbours": EditedNearestNeighbours,
    "NearMiss": NearMiss,
    "NeighbourhoodCleaningRule": NeighbourhoodCleaningRule,
    "OneSidedSelection": OneSidedSelection,
    "RandomUnderSampler": RandomUnderSampler,
    "RepeatedEditedNearestNeighbours": RepeatedEditedNearestNeighbours,
    "TomekLinks": TomekLinks,
    "None": DummySampler,
}

oversampler_list = list(oversampler.keys())
undersampler_list = list(undersampler.keys())

search_space = {"oversampling_method": oversampler_list, "undersampling_method": undersampler_list}

In [ ]:
def create_objective(create_model: Callable[[], BaseEstimator]) -> callable:
    def objective(trial: optuna.Trial) -> float:
        with mlflow.start_run(nested=True):
            oversampling_method = trial.suggest_categorical("oversampling_method", oversampler_list)
            undersammpling_method = trial.suggest_categorical(
                "undersampling_method",
                undersampler_list,
            )

            # Check if it has random_state parameter
            try:
                oversampler_instance = oversampler[oversampling_method](random_state=42)
            except TypeError:
                oversampler_instance = oversampler[oversampling_method]()

            try:
                undersampler_instance = undersampler[undersammpling_method](random_state=42)
            except TypeError:
                undersampler_instance = undersampler[undersammpling_method]()

            X_train_resampled, y_train_resampled = oversampler_instance.fit_resample(
                X_train, y_train
            )
            X_train_resampled, y_train_resampled = undersampler_instance.fit_resample(
                X_train_resampled, y_train_resampled
            )

            clf = create_model()

            clf.fit(X_train_resampled, y_train_resampled)
            y_pred = clf.predict(X_val)
            f1_score_val = f1_score(y_val, y_pred)

            mlflow.log_params(
                {
                    "oversampling_method": oversampling_method,
                    "undersampling_method": undersammpling_method,
                }
            )
            mlflow.log_metric("f1", f1_score_val)

        return f1_score_val

    return objective

## Decision Tree
First we will try a decision tree classifier to see how well it performs on the dataset for each sampler combination.

In [ ]:
objective = create_objective(lambda: DecisionTreeClassifier(random_state=42))

run_name = "Decision Tree Run"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
    )
    study.optimize(objective, n_trials=len(oversampler_list) * len(undersampler_list))

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_f1_score", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "stroke_prediction",
            "optimizer_engine": "optuna",
            "model_family": "DecisionTreeClassifier",
        }
    )

[I 2025-05-27 14:14:31,411] A new study created in memory with name: no-name-514c7f32-e413-427f-bff2-3c9639f5a3e0
[I 2025-05-27 14:14:32,943] Trial 0 finished with value: 0.13157894736842105 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RandomUnderSampler'}. Best is trial 0 with value: 0.13157894736842105.


🏃 View run powerful-mole-270 at: https://mlflow.spikehub.win/#/experiments/0/runs/7031bd829f3a498e9c031b7ff880ee18
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:14:34,466] Trial 1 finished with value: 0.0 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 0 with value: 0.13157894736842105.


🏃 View run languid-crane-137 at: https://mlflow.spikehub.win/#/experiments/0/runs/866075dc34064196912ebc74159d8a5e
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:14:34,869] Trial 2 finished with value: 0.1111111111111111 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'TomekLinks'}. Best is trial 0 with value: 0.13157894736842105.


...

🏃 View run likeable-grub-516 at: https://mlflow.spikehub.win/#/experiments/0/runs/ce7c98ec35c741719296c7a4911879ee
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:18:18,153] Trial 68 finished with value: 0.08955223880597014 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 29 with value: 0.22535211267605634.


🏃 View run youthful-slug-449 at: https://mlflow.spikehub.win/#/experiments/0/runs/bffaf1a89f7641479250def986b5ed8e
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:18:18,665] Trial 69 finished with value: 0.13513513513513514 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'NeighbourhoodCleaningRule'}. Best is trial 29 with value: 0.22535211267605634.


🏃 View run kindly-crow-927 at: https://mlflow.spikehub.win/#/experiments/0/runs/9c11eacf9771462ab5d6c736448119b4
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0
🏃 View run First Run at: https://mlflow.spikehub.win/#/experiments/1/runs/90c274b8d58d4fd49c54cf3db7405f47
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/1


## Random Forest
Next, we will try a random forest classifier to see if it can improve the performance over the decision tree and see if it compatitble with different sampler.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

search_space = {"oversampling_method": oversampler_list, "undersampling_method": undersampler_list}

objective = create_objective(lambda: RandomForestClassifier(random_state=42))

run_name = "Random Forest Run"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
    )
    study.optimize(objective, n_trials=len(oversampler_list) * len(undersampler_list))

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_f1_score", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "stroke_prediction",
            "optimizer_engine": "optuna",
            "model_family": "RandomForestClassifier",
        }
    )

[I 2025-05-27 14:31:24,124] A new study created in memory with name: no-name-221be009-18bd-4d43-917e-eee83b69569f
[I 2025-05-27 14:31:25,847] Trial 0 finished with value: 0.0 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RandomUnderSampler'}. Best is trial 0 with value: 0.0.


🏃 View run handsome-gull-843 at: https://mlflow.spikehub.win/#/experiments/0/runs/41ec6ef222dc401dafdfce0c74a06a9d
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:31:27,520] Trial 1 finished with value: 0.0 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 0 with value: 0.0.


🏃 View run bouncy-frog-638 at: https://mlflow.spikehub.win/#/experiments/0/runs/4a290fe223b44695a0cdb1ca99092b40
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:31:28,258] Trial 2 finished with value: 0.0 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'TomekLinks'}. Best is trial 0 with value: 0.0.


...

🏃 View run bald-hog-652 at: https://mlflow.spikehub.win/#/experiments/0/runs/c726c629bef64737871627b59652dba8
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:35:33,141] Trial 68 finished with value: 0.0 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 8 with value: 0.2545454545454545.


🏃 View run gaudy-crab-139 at: https://mlflow.spikehub.win/#/experiments/0/runs/815497af1fa749b4a45b4e39fabad1cf
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:35:34,203] Trial 69 finished with value: 0.0 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'NeighbourhoodCleaningRule'}. Best is trial 8 with value: 0.2545454545454545.


🏃 View run calm-donkey-726 at: https://mlflow.spikehub.win/#/experiments/0/runs/947bb90096b24b088a76c2fa405a15ff
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0
🏃 View run Random Forest Run at: https://mlflow.spikehub.win/#/experiments/1/runs/8df48cfb519d4010a3b3aacabd1f30cb
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/1


## K-Nearest Neighbors
Now, we will try a different model, K-Nearest Neighbors, to see if it can perform differently with the samplers.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

objective = create_objective(lambda: KNeighborsClassifier())

run_name = "Neighbors Classifier Run"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
    )
    study.optimize(objective, n_trials=len(oversampler_list) * len(undersampler_list))

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_f1_score", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "stroke_prediction",
            "optimizer_engine": "optuna",
            "model_family": "KNeighborsClassifier",
        }
    )

[I 2025-05-27 14:25:24,695] A new study created in memory with name: no-name-023785f1-9a09-461c-8d4b-099250af2560
[I 2025-05-27 14:25:26,157] Trial 0 finished with value: 0.047058823529411764 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RandomUnderSampler'}. Best is trial 0 with value: 0.047058823529411764.


🏃 View run kindly-goose-728 at: https://mlflow.spikehub.win/#/experiments/0/runs/1a4c64a98d774719866204f10dafe0ab
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:25:27,613] Trial 1 finished with value: 0.04938271604938271 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 1 with value: 0.04938271604938271.


🏃 View run omniscient-carp-834 at: https://mlflow.spikehub.win/#/experiments/0/runs/dc063dbe705846b3a9fe2183c0277b1d
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:25:28,071] Trial 2 finished with value: 0.16176470588235295 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'TomekLinks'}. Best is trial 2 with value: 0.16176470588235295.


...

🏃 View run secretive-cat-247 at: https://mlflow.spikehub.win/#/experiments/0/runs/793c89df4cc2418daf7f15c5b5a36636
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:29:16,110] Trial 68 finished with value: 0.17054263565891473 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 46 with value: 0.19330855018587362.


🏃 View run kindly-trout-592 at: https://mlflow.spikehub.win/#/experiments/0/runs/4547bbdca3184e57b5fffb8fcc0f2201
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:29:16,739] Trial 69 finished with value: 0.17543859649122806 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'NeighbourhoodCleaningRule'}. Best is trial 46 with value: 0.19330855018587362.


🏃 View run dapper-swan-283 at: https://mlflow.spikehub.win/#/experiments/0/runs/714052608d79456687fceb501a9d0a47
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0
🏃 View run Neighbors Classifier Run at: https://mlflow.spikehub.win/#/experiments/1/runs/f5cac5f679964f45b56fc41dd1ef216c
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/1


## CatBoost
Finally, we will try CatBoost classifier which we likely to use in production. We will see how it performs with the samplers and if it can outperform the previous models.

In [19]:
from catboost import CatBoostClassifier, Pool

val_data = Pool(
    data=X_val,
    label=y_val,
)

test_data = Pool(
    data=X_test,
    label=y_test,
)


def catboost_objective(trial: optuna.Trial) -> float:
    with mlflow.start_run(nested=True):
        oversampling_method = trial.suggest_categorical("oversampling_method", oversampler_list)
        undersammpling_method = trial.suggest_categorical(
            "undersampling_method",
            undersampler_list,
        )

        # Check if it has random_state parameter
        try:
            oversampler_instance = oversampler[oversampling_method](random_state=42)
        except TypeError:
            oversampler_instance = oversampler[oversampling_method]()

        try:
            undersampler_instance = undersampler[undersammpling_method](random_state=42)
        except TypeError:
            undersampler_instance = undersampler[undersammpling_method]()

        X_train_resampled, y_train_resampled = oversampler_instance.fit_resample(X_train, y_train)
        X_train_resampled, y_train_resampled = undersampler_instance.fit_resample(
            X_train_resampled, y_train_resampled
        )

        clf = CatBoostClassifier(
            random_seed=42,
            verbose=0,
            auto_class_weights="Balanced",
            task_type="GPU",
            eval_metric="F1",
        )

        clf.fit(X_train_resampled, y_train_resampled, eval_set=val_data, early_stopping_rounds=100)
        y_pred = clf.predict(X_val)
        f1_score_val = f1_score(y_val, y_pred)

        mlflow.log_params(
            {
                "oversampling_method": oversampling_method,
                "undersampling_method": undersammpling_method,
            }
        )
        mlflow.log_metric("f1", f1_score_val)

    return f1_score_val

In [20]:
run_name = "Catboost Classifier Run"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.GridSampler(search_space)
    )
    study.optimize(catboost_objective, n_trials=len(oversampler_list) * len(undersampler_list))

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_f1_score", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "stroke_prediction",
            "optimizer_engine": "optuna",
            "model_family": "CatBoostClassifier",
        }
    )

[I 2025-05-27 14:46:00,912] A new study created in memory with name: no-name-48fcd9d2-4d95-4388-918e-e504cde51ad6
[I 2025-05-27 14:46:05,733] Trial 0 finished with value: 0.1592920353982301 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RandomUnderSampler'}. Best is trial 0 with value: 0.1592920353982301.


🏃 View run smiling-bass-312 at: https://mlflow.spikehub.win/#/experiments/0/runs/a29898d1847d47728bc59bbba13ae1b3
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:46:09,902] Trial 1 finished with value: 0.17142857142857143 and parameters: {'oversampling_method': 'SMOTEN', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 1 with value: 0.17142857142857143.


🏃 View run bald-goat-942 at: https://mlflow.spikehub.win/#/experiments/0/runs/1c39f3519aeb4538a143b8ab785d0aef
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:46:13,185] Trial 2 finished with value: 0.25925925925925924 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'TomekLinks'}. Best is trial 2 with value: 0.25925925925925924.


...

🏃 View run auspicious-moth-915 at: https://mlflow.spikehub.win/#/experiments/0/runs/3d9fdea85a254ba0b8789b6b98686293
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:53:41,011] Trial 68 finished with value: 0.2716049382716049 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'RepeatedEditedNearestNeighbours'}. Best is trial 30 with value: 0.2972972972972973.


🏃 View run kindly-ox-227 at: https://mlflow.spikehub.win/#/experiments/0/runs/531bcdb3351a4314b85f318c57f957df
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0


[I 2025-05-27 14:53:45,815] Trial 69 finished with value: 0.3076923076923077 and parameters: {'oversampling_method': 'BorderlineSMOTE', 'undersampling_method': 'NeighbourhoodCleaningRule'}. Best is trial 69 with value: 0.3076923076923077.


🏃 View run classy-bat-656 at: https://mlflow.spikehub.win/#/experiments/0/runs/8b18908a396b440584f196cb9ede6feb
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/0
🏃 View run Catboost Classifier Run at: https://mlflow.spikehub.win/#/experiments/1/runs/dfb2ea7ea68a473f864673febe97bc6b
🧪 View experiment at: https://mlflow.spikehub.win/#/experiments/1


After testing the sampler we find the for the oversampling methods, the best one is the BorderlineSMOTE, and for the undersampling methods, the best one is the NeighbourhoodCleaningRule.

In [25]:
study.best_params, study.best_value

({'oversampling_method': 'BorderlineSMOTE',
  'undersampling_method': 'NeighbourhoodCleaningRule'},
 0.3076923076923077)